# Create an InterActive Graph of NTRL Lab Analysis Results

In [52]:
#import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from dateutil.parser import parse
from itertools import compress
from collections import OrderedDict
from datetime import date
import datetime
import re
from datetime import timedelta
import openpyxl
from sklearn.metrics import mean_squared_error
import plotly.express as px

import plotly.graph_objects as go
from plotly.subplots import make_subplots
# Create figure with secondary y-axis

In [2]:
#used in the analysis results data. cleans up the sampling time entry
def time_cleaner(x):
    time_regex = re.compile(r'[0-8a-zA-Z:]', re.IGNORECASE| re.VERBOSE|re.DOTALL)
    timeact = time_regex.findall(x)
    newtime = ''.join(timeact)
    t = parse(x)
    new_str_time = str(t.hour)+':'+str(t.minute)+':'+str(t.second)
    return new_str_time
def ave_filler(dfname, colname):
    dfcn = dfname[colname]
    try:        
        dfcn = pd.to_numeric(dfcn, errors='coerce')
    except:
        print('error found in' + str(colname))
        pass
    try:
        dfname[colname] = dfcn.where(dfcn.notnull(), other=(dfcn.fillna(method='ffill') + dfcn.fillna(method='bfill'))/2)
    except:
        print('nan not fixed' + str(colname))   
    return dfname
def generate_df_from_analysis(workbookTab, col_sampleDate, col_sampleTime, col_analysis, col_title):
    #get the start and end sample dates
    Smpl_day_index = list()
    Chem_Element_datelist = list()
    Chem_Element_list = list()
    Smpl_day_index = list()
    Chem_Element_datelist = list()
    Chem_Element_list = list()
    plus_oneday_list = ['0:0:0', '1:0:0', '2:0:0', '3:0:0', '4:0:0', '5:0:0']
    #gets the date day from the B column of the analysis file if there is value in the cell
    for i in range(1, workbookTab.max_row):
        if type(workbookTab[str(col_sampleDate) + str(i)].value) == int or type(workbookTab[str(col_sampleDate) + str(i)].value) == float:
            Smpl_day_index.append(i)
    #get the ending row that has date day
    for j in range(1, workbookTab.max_row):
        if type(workbookTab[str(col_sampleTime) + str(j)].value) == datetime.datetime:
            smpl_end_index = j
    for y in range(0, len(Smpl_day_index)):
        if Smpl_day_index[y] != Smpl_day_index[-1]:      
            for x in range(Smpl_day_index[y], Smpl_day_index[y+1]):
                if workbookTab[str(col_analysis)+str(x)].value is None: 
                    pass
                else:
                    if time_cleaner(str(workbookTab[str(col_sampleTime)+str(x)].value)) in plus_oneday_list:
                        date = str((currentmonthyr)+datetime.timedelta(workbookTab[str(col_sampleDate)+str(Smpl_day_index[y])].value))\
                        +' '+time_cleaner(str(workbookTab[str(col_sampleTime)+str(x)].value))
                        Chem_Element_datelist.append(parse(date))  
                        chem_element = workbookTab[str(col_analysis)+str(x)].value
                        Chem_Element_list.append(chem_element)
                    else:
                        date = str((currentmonthyr)-datetime.timedelta(1)+datetime.timedelta(workbookTab[str(col_sampleDate)+str(Smpl_day_index[y])].value))\
                        +' '+time_cleaner(str(workbookTab[str(col_sampleTime)+str(x)].value))
                        Chem_Element_datelist.append(parse(date))  
                        chem_element = workbookTab[str(col_analysis)+str(x)].value
                        Chem_Element_list.append(chem_element)

        if Smpl_day_index[y] == Smpl_day_index[-1]:
            for x in range(Smpl_day_index[y], smpl_end_index+1):
                if workbookTab[str(col_analysis)+str(x)].value is None:   
                    pass
                else:
                    if time_cleaner(str(workbookTab[str(col_sampleTime)+str(x)].value)) in plus_oneday_list:
                        date = str((currentmonthyr)+datetime.timedelta(workbookTab[str(col_sampleDate)+str(Smpl_day_index[y])].value))\
                        +' '+time_cleaner(str(workbookTab[str(col_sampleTime)+str(x)].value))
                        Chem_Element_datelist.append(parse(date))
                        chem_element = workbookTab[str(col_analysis)+str(x)].value
                        Chem_Element_list.append(chem_element)
                    else:
                        date = str((currentmonthyr)-datetime.timedelta(1)+datetime.timedelta(workbookTab[str(col_sampleDate)+str(Smpl_day_index[y])].value))\
                        +' '+time_cleaner(str(workbookTab[str(col_sampleTime)+str(x)].value))
                        Chem_Element_datelist.append(parse(date))
                        chem_element = workbookTab[str(col_analysis)+str(x)].value
                        Chem_Element_list.append(chem_element)

    Chem_Element_list_pair = list(zip(Chem_Element_list, Chem_Element_datelist[0:]))
    Chem_Element_df = pd.DataFrame(Chem_Element_list_pair)
    Chem_Element_df.columns = [col_title, 'DATETIME'] ##check if list title is updated
    Chem_Element_df.set_index('DATETIME', inplace=True)

    return Chem_Element_df

def generate_add_trace(figname, df_elemental, axis_bool):
    colname = df_elemental.columns[0]
    figname.add_trace(go.Scatter(x=df_elemental.index, y=df_elemental[colname], name=colname), secondary_y=axis_bool,)
#---------------------------------------


In [3]:
# store the Filepath of the shutdown schedule excel file
#shutdown_filepath = r'\\THGOBNA002\thpal\05 Production\260 Tech - Shutdown\Shutdown 202009 Major Shutdown\Process B\00 Schedule\2020 Major SD Schedule Rev 2.4.xlsx'
shutdown_filepath = r'\\THGOBNA002\thpal\05 Production\260 Tech - Shutdown\Shutdown 202009 Major Shutdown\H2S\001 Schedule\H2S SEPTEMBER SCHEDULE REV.06.xlsx'

In [4]:
#-------------------------------------------------------------------------------------------------------------------------#
# Load the shutdown schedule excel file
raw_shutdown_sched = pd.read_excel(shutdown_filepath, sheet_name='H2S Oct 2020 SD Rev.06', index_col=False)


In [16]:
raw_shutdown_sched.iloc[4][raw_shutdown_sched.columns[:5]]

Unnamed: 0    TAG NUMBER
Unnamed: 1      ACTIVITY
Unnamed: 2         START
Unnamed: 3        FINISH
Unnamed: 4      DURATION
Name: 4, dtype: object

In [30]:
shutdown_sched_2 = shutdown_sched_2.drop('index', axis=1)

In [41]:
shutdown_sched_1 = raw_shutdown_sched.iloc[27:][raw_shutdown_sched.columns[:5]]
shutdown_sched_1.columns = raw_shutdown_sched.iloc[4][raw_shutdown_sched.columns[:5]]
shutdown_sched_2 = shutdown_sched_1.rename(columns ={'TAG NUMBER': 'Equipment', 'ACTIVITY':'Activities', 'DURATION':'Duration_hrs_plan'}).reset_index().drop('index', axis=1)

shutdown_sched_2['Equipment'] = shutdown_sched_2['Equipment'].fillna(method='ffill') ## FILL THE BLANK CELLS IN THE EQUIPMENT COLUMN USING THE FFILL METHOD.

shutdown_sched_2['DATETIME'] = shutdown_sched_2['START'] ## CREATE A DATETIME COLUMN EQUAL TO THE START COLUMN. POSSIBLE WE'LL CHANGE THIS TO USING THE FINISH COLUMN

shutdown_sched_3 = shutdown_sched_2.dropna().set_index('DATETIME') ## DROP THE EMPTY ROWS. THEN SET THE INDEX TO DATETIME. THIS TELLS US WHEN THE ACTIVITY IS SCHEDULED TO START. OR WE CAN CHANGE IT TO FINISH.

In [45]:
# CREATE A SEPARATE DF

shutdown_sched_4 = shutdown_sched_3.rename(columns={"START":'START_plan','FINISH':'FINISH_plan'})
shutdown_sched_4['START_act'] = 0
shutdown_sched_4['FINISH_act'] = 0
#shutdown_sched_4 = shutdown_sched_4.drop(['DATETIME'], axis=1)

shutdown_sched_4['Duration_hrs_act'] = 0
shutdown_sched_4['Complete_or_Not'] = 0

#shutdown_sched_4 = shutdown_sched_4.drop(['level_0','index'], axis=1)
#shutdown_sched_4 = shutdown_sched_4.reset_index()

In [47]:
# CREATE A LIST CONTAINING ALL UNIQUE ENTRIES IN THE EQUIPMENT COLUMN
List_equiments = shutdown_sched_4['Equipment'].unique()
List_equiments_rshp = List_equiments.reshape(len(List_equiments), 1) # array of all equipments shaped to fit in one column


#### Create a daterange for the entire shudown duration

In [50]:
#### Create a daterange for the entire shudown duration
from datetime import datetime
#Create an array of datetime as range of datetime incremented hourly for the entire shutdown duration
l = [datetime(2020, 10, 22, 0, 0),
    datetime(2020, 10, 23, 0, 0),
    datetime(2020, 10, 24, 0, 0),
    datetime(2020, 10, 25, 0, 0),
    datetime(2020, 10, 26, 0, 0),
    datetime(2020, 10, 27, 0, 0),
    datetime(2020, 10, 28, 0, 0),
    datetime(2020, 10, 29, 0, 0),
    datetime(2020, 10, 30, 0, 0),
    datetime(2020, 10, 31, 0, 0),
    datetime(2020, 11, 1, 0, 0),
    datetime(2020, 11, 2, 0, 0),
    datetime(2020, 11, 3, 0, 0),
    datetime(2020, 11, 4, 0, 0),
    datetime(2020, 11, 5, 0, 0),
    datetime(2020, 11, 6, 0, 0),
    datetime(2020, 11, 7, 0, 0),
    datetime(2020, 11, 8, 0, 0),
    datetime(2020, 11, 9, 0, 0),
    datetime(2020, 11, 10, 0, 0),
    datetime(2020, 11, 11, 0, 0)]

hours = np.arange(0,24)

t = [[item.replace(hour=hour) for hour in hours] for item in l]
u = np.array(t)
Shutdown_date_range = u.reshape((504,1))
#print(Shutdown_date_range)
#print(len(Shutdown_date_range))
#daterange code END--------------------


In [188]:
# code 1-A
percent_prog_act_Plan_arr = np.zeros((len(shutdown_sched_4['Activities']),len(Shutdown_date_range_1x504.T)))

#percent_prog_act_Plan_arr = np.zeros((len(shutdown_sched_4['Activities']),1))

Shutdown_date_range_1x504 = u.reshape((1,504)) # u is from the daterange code block

print("percent_Prog shape: {}.\nShutdowndate range shape: {}".format(Shutdown_date_range_1x504.shape, percent_prog_act_Plan_arr.shape))
# Continuation of code 1-A

for act in range(0, len(shutdown_sched_4['Activities'])):

    activity_name = shutdown_sched_4.iloc[act]['Activities']
    time_start = shutdown_sched_4.iloc[act]['START_plan']
    duration = shutdown_sched_4.iloc[act]['Duration_hrs_plan']
    
    array_time_diff = Shutdown_date_range_1x504 - time_start # Creates an array of the difference between the datetime reference and the planned start of the activity
    
    if duration != 0: # check if the duration is not set to zero

        planned_progress_per_act = 100*array_time_diff/datetime.timedelta(hours=duration)
        print("planned_progress_per_act shape: {}".format(planned_progress_per_act.shape))

    else:
        planned_progress_per_act = np.zeros((1,len(Shutdown_date_range_1x504.T)))
        
    #print("percent_prog_act_Plan_arr shape: {}".format(percent_prog_act_Plan_arr.shape))
    percent_prog_act_Plan_arr[act] = planned_progress_per_act #store the calculated percent progress into the array called percent_prog_act_Plan_arr

    
    
df_hourly_progress_plan = pd.DataFrame(percent_prog_act_Plan_arr)
dict_progress_col_SDdate = dict(zip(df_hourly_progress_plan.columns, Shutdown_date_range_1x504[0].T)) # Note the use of the transpose function T    

percent_Prog shape: (1, 504).
Shutdowndate range shape: (301, 504)
planned_progress_per_act shape: (1, 504)
planned_progress_per_act shape: (1, 504)
planned_progress_per_act shape: (1, 504)
planned_progress_per_act shape: (1, 504)
planned_progress_per_act shape: (1, 504)
planned_progress_per_act shape: (1, 504)
planned_progress_per_act shape: (1, 504)
planned_progress_per_act shape: (1, 504)
planned_progress_per_act shape: (1, 504)
planned_progress_per_act shape: (1, 504)
planned_progress_per_act shape: (1, 504)
planned_progress_per_act shape: (1, 504)
planned_progress_per_act shape: (1, 504)
planned_progress_per_act shape: (1, 504)
planned_progress_per_act shape: (1, 504)
planned_progress_per_act shape: (1, 504)
planned_progress_per_act shape: (1, 504)
planned_progress_per_act shape: (1, 504)
planned_progress_per_act shape: (1, 504)
planned_progress_per_act shape: (1, 504)
planned_progress_per_act shape: (1, 504)
planned_progress_per_act shape: (1, 504)
planned_progress_per_act shape:

planned_progress_per_act shape: (1, 504)
planned_progress_per_act shape: (1, 504)
planned_progress_per_act shape: (1, 504)
planned_progress_per_act shape: (1, 504)
planned_progress_per_act shape: (1, 504)
planned_progress_per_act shape: (1, 504)
planned_progress_per_act shape: (1, 504)
planned_progress_per_act shape: (1, 504)
planned_progress_per_act shape: (1, 504)
planned_progress_per_act shape: (1, 504)
planned_progress_per_act shape: (1, 504)
planned_progress_per_act shape: (1, 504)
planned_progress_per_act shape: (1, 504)
planned_progress_per_act shape: (1, 504)
planned_progress_per_act shape: (1, 504)
planned_progress_per_act shape: (1, 504)
planned_progress_per_act shape: (1, 504)
planned_progress_per_act shape: (1, 504)
planned_progress_per_act shape: (1, 504)
planned_progress_per_act shape: (1, 504)
planned_progress_per_act shape: (1, 504)
planned_progress_per_act shape: (1, 504)
planned_progress_per_act shape: (1, 504)
planned_progress_per_act shape: (1, 504)
planned_progress

In [189]:
# code 2
shutdown_sched_4 = shutdown_sched_4[['Equipment', 'Activities', 'START_plan', 'FINISH_plan',
       'Duration_hrs_plan', 'START_act', 'FINISH_act', 'Duration_hrs_act',
       'Complete_or_Not']]


# Store the array of hourly shutdown progress as columns in the shutdown_sched_4 dataframe
for i in range(len(dict_progress_col_SDdate)):
    shutdown_sched_4['%Progress_Plan_asOf_{}'.format(dict_progress_col_SDdate[i])] = percent_prog_act_Plan_arr.T[i]
    
    # replace the values in the percent progress to a range 0 to 100
    colname_var = '%Progress_Plan_asOf_{}'.format(dict_progress_col_SDdate[i])
    shutdown_sched_4[colname_var] = np.where(shutdown_sched_4[colname_var]>100, 100, np.where(shutdown_sched_4[colname_var]<0, 0, shutdown_sched_4[colname_var]))
    

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]


In [421]:
# CODE 3
### COUNTING THE TOTAL HOURLY PROGRESS PER EQUIPMENT ###
### NOT INCLUDING THE MAINTENANCE ACTIVITIES ###

progress_plan_columns = shutdown_sched_4.columns[-504:] # the list of columns containing the %progress per activity as planned

List_equiments = shutdown_sched_4['Equipment'].unique() # list of the unique activities in the shutdown_sched_4 dataframe
List_equiments_rshp = List_equiments.reshape(len(List_equiments), 1) # array of all equipments shaped to fit in one column

# Initiallize the variables with zeros
SUM_percent_progress_of_activity_array = 0 #np.zeros((1, len(progress_plan_columns))) # the array containing all the %progress for a single equipment for the entire duration of the shutdown
SUM_percent_progress_of_equipment_array = np.zeros((len(List_equiments), len(progress_plan_columns))) # the array containing all the %progress for all the equipments for the entire duration of the shutdown


for eqpmnt_index in range(len(List_equiments_rshp)): #loop through the list of equipments
    eq = List_equiments_rshp[eqpmnt_index][0] # get a single name of the equipment from the list of equipments
    print(eq) # Print the equipment taken the list and check it in the output for confirmation
    count = shutdown_sched_4[(shutdown_sched_4['Equipment'] == eq)].count() # this counts the total number of activities under the same equipment. this includes the maintenance activities
    
    total_activities_count = 0 # initiallize the total_activities counter to zero. this counts all the activities prior to maintenance handover or inspection. so this counts the prodn activts leading up to handover
    percent_progress_of_activity_array = 0 # initiallize to zero. this will store an array of activity progress for the entire shutdown daterange
    SUM_percent_progress_of_activity_array = 0 # initiallize to zero. this will store an array of the percent progress of all the activities for the same equipment for the entire shutdown daterange
    for act in range(0, len(shutdown_sched_4[shutdown_sched_4['Equipment'] == eq])): # loop thru all the rows with the same equipment name eq
        
        activity_name = shutdown_sched_4[shutdown_sched_4['Equipment'] == eq]['Activities'].iloc[act]   # get the activity name one by one
        percent_progress_of_activity_array = shutdown_sched_4[shutdown_sched_4['Equipment'] == eq][progress_plan_columns].iloc[act]   # get the entire array of the progress for entire SD daterange
        
        SUM_percent_progress_of_activity_array = SUM_percent_progress_of_activity_array + percent_progress_of_activity_array # add up all the progress of all the activities for the same equipment

        print(activity_name)
        match = re.search(r'\binspection\b',activity_name.lower()) # check the activity for the keyword 'inspection'
        match1 = re.search(r'\bmaintenance\b',activity_name.lower()) # check the activity for the keyword 'maintenance'
        if (not match) and (not match1) : # check if there is no match
            #print(activity_name)
            total_activities_count+=1
        else:
            print("index: {}, activitiy: {}".format(total_activities_count, activity_name))
            break # stop looping on the same equipment if the activity matches the keywords 'inspection' or 'maintenance'
    
    equipment_percent_progress_total = 100*total_activities_count
    SUM_percent_progress_of_equipment_array[eqpmnt_index] = 100*SUM_percent_progress_of_activity_array/equipment_percent_progress_total
       
df_prcnt_progress = pd.DataFrame(SUM_percent_progress_of_equipment_array)
df_prcnt_progress['equipment'] = List_equiments_rshp



De-Zn Feedline
104PU01 Stop
LOTO Installation (Valves & Pump)
Feedline level lowering/105TK01 Inerting
Feed Line Blinding/105TK01 inlet
Feed Line Draining
Handover to Maintenance
index: 5, activitiy: Handover to Maintenance
105TK01
De-Zn Over Reaction 
Shutdown
Feedline level lowering/105TK01 Inerting
Emptying
Feed Line Blinding/105TK01 inlet
Inerting (106SR02)
Blinding
M/H Opening
Eductor/Lighting Installation
Tank Clean-up (Dead load recovery)
   > Siphoning
   > Vacuuming
   > Scales removal & nozzles descaling
   > HDPE drain pipes draining
Handover to Mechanical/Initial Inspection
index: 14, activitiy: Handover to Mechanical/Initial Inspection
105TK02
De-Zn Over Reaction 
Shutdown
Emptying
Inerting
Blinding
M/H Opening
Eductor/Lighting Installation
Tank Clean-up (Dead load recovery)
   > Siphoning
   > Vacuuming
   > Scales removal & nozzles descaling
   > HDPE drain pipes draining
Handover to Mechanical/Initial Inspection
index: 12, activitiy: Handover to Mechanical/Initial Inspe

Deadload Recovery
Inside Clean-up
106AT004 RAM source LOTO installation
Handover to Maintenance
index: 9, activitiy: Handover to Maintenance
106FT02A
Clean-up (including conveyors)
Handover to Maintenance
index: 1, activitiy: Handover to Maintenance
106FT02B
Clean-up (including conveyors)
Handover to Maintenance
index: 1, activitiy: Handover to Maintenance
106TK08
Emptying
Blinding
MH Opening
Lighting/Eductor Installation
Inside Clean-up
Maintenance Inspection and H/O
index: 5, activitiy: Maintenance Inspection and H/O
106TK04
Dilution
Emptying
Draining
LOTO installation (Pump, blower, agitator, valves)
Inerting to common vent
Isolation from Common Vent
MH Opening
Lighting/Eductor Installation
Inside Clean-up
Maintenance Inspection and H/O
index: 9, activitiy: Maintenance Inspection and H/O
106TK05
Dilution
Emptying (Seed line flushing)
Draining
Inerting to common vent
Isolation from Common Vent
Inerting to 106SR03
Blinding
MH Opening
Lighting/Eductor Installation
Inside Clean-up
Maint

In [193]:
# CODE 3
### COUNTING THE TOTAL HOURLY PROGRESS PER EQUIPMENT ###
### INCLUDING ALL ACTIVITIES ###

progress_plan_columns = shutdown_sched_4.columns[-504:] # the list of columns containing the %progress per activity as planned

List_equiments = shutdown_sched_4['Equipment'].unique() # list of the unique activities in the shutdown_sched_4 dataframe
List_equiments_rshp = List_equiments.reshape(len(List_equiments), 1) # array of all equipments shaped to fit in one column

# Initiallize the variables with zeros
SUM_percent_progress_of_activity_array = 0 #np.zeros((1, len(progress_plan_columns))) # the array containing all the %progress for a single equipment for the entire duration of the shutdown
SUM_percent_progress_of_equipment_array = np.zeros((len(List_equiments), len(progress_plan_columns))) # the array containing all the %progress for all the equipments for the entire duration of the shutdown


for eqpmnt_index in range(len(List_equiments_rshp)): #loop through the list of equipments
    eq = List_equiments_rshp[eqpmnt_index][0] # get a single name of the equipment from the list of equipments
    print(eq) # Print the equipment taken the list and check it in the output for confirmation
    count = shutdown_sched_4[(shutdown_sched_4['Equipment'] == eq)].count() # this counts the total number of activities under the same equipment. this includes the maintenance activities
    
    total_activities_count = 0 # initiallize the total_activities counter to zero. this counts all the activities prior to maintenance handover or inspection. so this counts the prodn activts leading up to handover
    percent_progress_of_activity_array = 0 # initiallize to zero. this will store an array of activity progress for the entire shutdown daterange
    SUM_percent_progress_of_activity_array = 0 # initiallize to zero. this will store an array of the percent progress of all the activities for the same equipment for the entire shutdown daterange
            
    for act in range(0, len(shutdown_sched_4[shutdown_sched_4['Equipment'] == eq])): # loop thru all the rows with the same equipment name eq
        
        activity_name = shutdown_sched_4[shutdown_sched_4['Equipment'] == eq]['Activities'].iloc[act]   # get the activity name one by one
        percent_progress_of_activity_array = shutdown_sched_4[shutdown_sched_4['Equipment'] == eq][progress_plan_columns].iloc[act]   # get the entire array of the progress for entire SD daterange
        
        SUM_percent_progress_of_activity_array = SUM_percent_progress_of_activity_array + percent_progress_of_activity_array # add up all the progress of all the activities for the same equipment

        total_activities_count+=1
        print("index: {}, activity: {}".format(total_activities_count, activity_name))
    
    equipment_percent_progress_total = 100*total_activities_count
    SUM_percent_progress_of_equipment_array[eqpmnt_index] = 100*SUM_percent_progress_of_activity_array/equipment_percent_progress_total
       
df_prcnt_progress_MAINTE_INCLD = pd.DataFrame(SUM_percent_progress_of_equipment_array)
df_prcnt_progress_MAINTE_INCLD['equipment'] = List_equiments_rshp


# Part of temporary code

###

df_Shutdown_date_range = pd.DataFrame(Shutdown_date_range) # Create dataframe from Shutdown_date_range array
df_prcnt_progress_MAINTE_INCLD_transposed = df_prcnt_progress_MAINTE_INCLD.T # Transpose the df_prcnt_progress dataframe
equiment_list = df_prcnt_progress_MAINTE_INCLD_transposed.iloc[-1] # get the list of equipments by slicing the last row of the df_prcnt_progress_transposed dataframe
df_prcnt_progress_MAINTE_INCLD_transposed = df_prcnt_progress_MAINTE_INCLD_transposed.drop('equipment')  # drop the row with equipment
df_prcnt_progress_MAINTE_INCLD_transposed['DATETIME'] = df_Shutdown_date_range[0] # Create a column called the DATETIME using the values from df_Shutdown_date_range
df_prcnt_progress_MAINTE_INCLD_transposed = df_prcnt_progress_MAINTE_INCLD_transposed.set_index('DATETIME') # set the DATETIME as the index
df_prcnt_progress_MAINTE_INCLD_transposed.columns = equiment_list # rename the columns with the equipment list using the equipment_list variable


# replace the values in the df_prcnt_progress_transposed to a range 0 to 100
for col in range(len(df_prcnt_progress_MAINTE_INCLD_transposed.columns)):
    column = df_prcnt_progress_MAINTE_INCLD_transposed.columns[col]
    df_prcnt_progress_MAINTE_INCLD_transposed[column] = np.where(df_prcnt_progress_MAINTE_INCLD_transposed[column]>=100, 100, 
                                                                 np.where(df_prcnt_progress_MAINTE_INCLD_transposed[column]<0, 0, df_prcnt_progress_MAINTE_INCLD_transposed[column]))
    
    


H2  
index: 1, activity: Operation/Shutdown
index: 2, activity: PSA Depressurization & Isolation for E&I Activity
index: 3, activity: Handover to Maintenance - E&I
index: 4, activity: H2 Plant Valves Diaphragm Replacement
index: 5, activity: 421VE10D System Inerting
index: 6, activity: 421VE10D System Depressurization
index: 7, activity: 421HV013 and 421PV030 Replacement
index: 8, activity: 421VE10 Leak Checking and Pressure Hold
index: 9, activity: 421VE10 System Inerting for Start-up
index: 10, activity: Methanol Line Draining
index: 11, activity: 421HV017/018/019/020 Pull-out and Inspection
index: 12, activity: Cooling Water Line Draining
index: 13, activity: 421FS007 Pull-out, Inspection, Clean-up
index: 14, activity: Handover to Maintenance - Mechanical
index: 15, activity: Demineralized Water Line Draining
index: 16, activity: Demineralized Water Line Rectification/Spool Modification
index: 17, activity: Demineralized Water Line Leak Checking/Rectification/Online
index: 18, activ

index: 9, activity: UTG/DPT
index: 10, activity: Final Inspection and Clean-up
index: 11, activity: Bucket Strainer Reinstatement
index: 12, activity: Box-up/De-blinding
Train 2 - General
index: 1, activity: Spare Time for Maintenance/Production Activities
index: 2, activity: 209 System Leak Checking
index: 3, activity: 209 System Inerting
index: 4, activity: 209 System Blind Flange Installation
index: 5, activity: SIF Testing
index: 6, activity: 209HX12 Preparation and 209VE01 Fill-up
index: 7, activity: 209VE01 Heat-up with N2 Bubbling
index: 8, activity: Start-up Checksheet 
index: 9, activity: T-209 H2S Reaction Start
421VP02/DR01AB
index: 1, activity: Operation
index: 2, activity: Shutdown
index: 3, activity: Start-up
525NG02
index: 1, activity: 530CT31 Operation
index: 2, activity: 530CT31 Shutdown and Handover
index: 3, activity: 530CT31 Start-up
index: 4, activity: 525NG01 Operation
index: 5, activity: 525NG01 Shutdown and Handover
index: 6, activity: 525NG01 Start-up (50% Oper

In [207]:
# prepare a list of columns for use in summing up progress per area    
columns_H2_N2_MeOH = ['H2  ', 'MeOH', '421VP02/DR01AB', '525NG02']
columns_Scrubbers = ['109VE11/\nPSV', '109SR01/02', '109SR03', '109SR04']
columns_Train1 = ['109VE01', '109HX12/13', '109HX11', '109VE02', '109VE03', 'Train 1 - General']
columns_Train2 = ['209VE01', '209HX12/13', '209HX11', '209VE02', '209VE03', 'Train 2 - General']
columns_Sulfur_steam_ProdHeader = ['109FT01AB', '109TK01/02', '109TK03', 'Common Product Header', 'Package Boiler/ Steam']


# Compute the average percent progress per area in H2S plant
progress_plan_MAINTE_H2_N2_MeOH = np.mean(df_prcnt_progress_MAINTE_INCLD_transposed[columns_H2_N2_MeOH], axis=1) #/len(columns_MS)
progress_plan_MAINTE_Scrubbers = np.mean(df_prcnt_progress_MAINTE_INCLD_transposed[columns_Scrubbers], axis=1) #/len(columns_dezn)
progress_plan_MAINTE_Train1 = np.mean(df_prcnt_progress_MAINTE_INCLD_transposed[columns_Train1], axis=1) #/len(columns_108_chem)
progress_plan_MAINTE_Train2 = np.mean(df_prcnt_progress_MAINTE_INCLD_transposed[columns_Train2], axis=1) #/len(columns_111_ls)
progress_plan_MAINTE_Header_sulfur_steam = np.mean(df_prcnt_progress_MAINTE_INCLD_transposed[columns_Sulfur_steam_ProdHeader], axis=1) #/len(columns_dezn)

# Create a column for each area and then store the series containing the percent progress of each area
df_prcnt_progress_MAINTE_INCLD_transposed['H2_N2_MeOH'] = progress_plan_MAINTE_H2_N2_MeOH
df_prcnt_progress_MAINTE_INCLD_transposed['Scrubbers'] = progress_plan_MAINTE_Scrubbers
df_prcnt_progress_MAINTE_INCLD_transposed['Train1'] = progress_plan_MAINTE_Train1
df_prcnt_progress_MAINTE_INCLD_transposed['Train2'] = progress_plan_MAINTE_Train2
df_prcnt_progress_MAINTE_INCLD_transposed['Sulfur_steam_ProdHeader'] = progress_plan_MAINTE_Header_sulfur_steam

#### Create the plot

In [285]:

#### Plotting the SD progress including the maintenance activities
#
fig_SDH2S_MAINTE = make_subplots(
    rows=1, cols=1,
    specs=[[{"type": "scatter"}]])
fig_SDH2S_MAINTE.add_trace(go.Scatter(x=df_prcnt_progress_MAINTE_INCLD_transposed['H2_N2_MeOH'].index, y=df_prcnt_progress_MAINTE_INCLD_transposed['H2_N2_MeOH'], name='H2_N2_MeOH_plan'), secondary_y=False,)
fig_SDH2S_MAINTE.add_trace(go.Scatter(x=df_prcnt_progress_MAINTE_INCLD_transposed['Scrubbers'].index, y=df_prcnt_progress_MAINTE_INCLD_transposed['Scrubbers'], name='Scrubbers_plan'), secondary_y=False,)     
fig_SDH2S_MAINTE.add_trace(go.Scatter(x=df_prcnt_progress_MAINTE_INCLD_transposed['Train1'].index, y=df_prcnt_progress_MAINTE_INCLD_transposed['Train1'], name='Train1_plan'), secondary_y=False,)    
fig_SDH2S_MAINTE.add_trace(go.Scatter(x=df_prcnt_progress_MAINTE_INCLD_transposed['Train2'].index, y=df_prcnt_progress_MAINTE_INCLD_transposed['Train2'], name='Train2_plan'), secondary_y=False,)    
fig_SDH2S_MAINTE.add_trace(go.Scatter(x=df_prcnt_progress_MAINTE_INCLD_transposed['Sulfur_steam_ProdHeader'].index, y=df_prcnt_progress_MAINTE_INCLD_transposed['Sulfur_steam_ProdHeader'], name='Sulfur_steam_ProdHeader_plan'), secondary_y=False,)     


# adding a range slider below the graph in plotly
#Example:
#title_plotAcid = 'SD Progress 2020'
#fig_SD.update_layout(title_text=title_plotAcid,xaxis=dict(rangeslider=dict(visible = True),type='date'))
fig_SDH2S_MAINTE.update_xaxes(title_text="Datetime") # Set x-axis title
fig_SDH2S_MAINTE.update_yaxes(title_text="ShutDown Progress %",secondary_y=False) # Set y-axes titles primary axis


title_plot_SD = "<b>H2S PLANT  Shutdown Progress</b> Hourly Tracker"
fig_SDH2S_MAINTE.update_layout(title_text=title_plot_SD ,hovermode="x",
                       hoverdistance=100, # Distance to show hover label of data point
                       spikedistance=1000, # Distance to show spike
                       xaxis=dict(linecolor="#BCCCDC",
                                   showspikes=True, # Show spike line for X-axis
                                   # Format spike
                                   spikethickness=2,
                                   spikedash="dot",
                                   spikecolor="#999999",
                                   spikemode="across"), font=dict(size=10)
                               )

# Mark the datetime of power limitation by shading these regions with light color
fig_SDH2S_MAINTE.add_shape(
        # filled Rectangle
            type="rect",
            x0='2020-11-01 04:30:00',
            y0=0,
            x1='2020-11-01 19:00:00',
            y1=100,
            fillcolor="pink",
            opacity=0.5,
            layer="below",
            line_width=0,
        )
fig_SDH2S_MAINTE.add_shape(
        # filled Rectangle
            type="rect",
            x0='2020-11-08 16:00:00',
            y0=0,
            x1='2020-11-09 18:00:00',
            y1=100,
            fillcolor="LightSalmon",
            opacity=0.5,
            layer="below",
            line_width=0,
        )

fig_SDH2S_MAINTE.add_shape(
        # filled Rectangle
            type="rect",
            x0='2020-10-23 14:00:00',
            y0=0,
            x1='2020-10-25 08:00:00',
            y1=100,
            fillcolor="LightSalmon",
            opacity=0.5,
            layer="below",
            line_width=0,
        )


fig_SDH2S_MAINTE.update_shapes(dict(xref='x', yref='y'))

fig_SDH2S_MAINTE.add_annotation(
        x='2020-10-24 10:00:00',
        xref="x",
        yref="paper",
        text="Power Limitation",
        textangle=-90,
        yanchor="middle",
        showarrow=False,
        font=dict(
            family="Courier New, monospace",
            size=16,
            color="#8e2800",
            
            ),
        ax=20,
        ay=-30,
        opacity=0.5
        )

fig_SDH2S_MAINTE.add_annotation(
        x='2020-11-09 05:00:00',
        xref="x",
        yref="paper",
        text="Power Limitation",
        textangle=-90,
        yanchor="middle",
        showarrow=False,
        font=dict(
            family="Courier New, monospace",
            size=16,
            color="#8e2800",
            
            ),
        ax=20,
        ay=-30,
        opacity=0.5
        )

fig_SDH2S_MAINTE.add_annotation(
        x='2020-11-01 12:00:00',
        xref="x",
        yref="paper",
        text="E&I Blackout Activity",
        textangle=-90,
        yanchor="middle",
        showarrow=False,
        font=dict(
            family="Courier New, monospace",
            size=16,
            color="#850017",
            
            ),
        ax=20,
        ay=-30,
        opacity=0.5
        )

#### Export the plot as html

In [294]:
plt.close()
with open(r'\\THGOBNA002\thpal\05 Production\150 DCS\Miscellaneous\MS_Progress_of_Equipment_Handover_Tracker.html', 'a') as f:
    f.write(fig_SDH2S_MAINTE.to_html(fig_SDH2S_MAINTE, full_html=True, include_plotlyjs='cdn'))
    
with open(r'C:\Users\v.t.flores\Documents\MS_Progress_of_Equipment_Handover_Tracker_actual_temporary.html', 'a') as f:
    f.write(fig_SDH2S_MAINTE.to_html(fig_SDH2S_MAINTE, full_html=True, include_plotlyjs='cdn'))


In [295]:
plt.close()
with open(r'C:\Users\v.t.flores\Documents\MS_Progress_of_Equipment_Handover_Tracker.html', 'a') as f:
    f.write(fig_SDH2S_MAINTE.to_html(fig_SDH2S_MAINTE, full_html=True, include_plotlyjs='cdn'))